In [1083]:
%pylab inline

import os
import unicodedata, re
from PIL import Image
from xml.etree import ElementTree as ET
Image.MAX_IMAGE_PIXELS = None

import logging as log
log.getLogger().setLevel(log.INFO)

import pandas

Populating the interactive namespace from numpy and matplotlib


/usr/lib/python3.9/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['f', 'log', 'style']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [1084]:
overrides = {"0 -PLaNCK-PLANMAP_GEOMAPPING-OTHER-WORLDS.png": {"href": "https://www.planck-magazine.it/le-uscite/?titolo=geomapping-other-worlds&id=0&preview=y"}}

In [1085]:
from pathvalidate import sanitize_filename, sanitize_filepath

from pathlib import Path

excel_file = Path("./captions.xlsx")
if excel_file.exists():
    init_df = pandas.read_excel(excel_file)
else:
    init_df = pandas.DataFrame()

In [1086]:
# files_folder = "/home/luca/Documenti/planmap/website_data/original/bianca/"


outfolder = "/home/luca/Documenti/planmap/website_data/artworks/"
dataurl = "https://planmap.eu/sites/default/files/artworks/"

# dataurl = outfolder



In [1087]:
def find_all_images(folder):
    import glob
    jpgs = glob.glob(folder + "/*.jpg")
    pngs = glob.glob(folder + "/*.png")

    images = jpgs + pngs
    
    return sort(images)



In [1088]:
a = "a"
a.upper()

'A'

In [1089]:
def resize_image(file, outfile, maxsize=(512, 512), write=True, toformat="png"):
    if outfile[-4:].lower() != toformat:
        log.debug("renaming extensions to png")
        outfile = outfile[:-3] + toformat    
    if write:
        im = Image.open(file).convert("RGB")
        im.thumbnail(maxsize, Image.ANTIALIAS)
        
        if toformat in ["jpg", "jpeg"]:
            toformat = "jpeg"
        
        im.save(outfile, toformat.upper(), quality=80)
        
    return outfile # actual written file

In [1090]:
import pathlib
pathlib.Path("/home/luca/Documenti/planmap/website_data/original/bianca/0 -PLaNCK-PLANMAP_GEOMAPPING-OTHER-WORLDS.png").name

'0 -PLaNCK-PLANMAP_GEOMAPPING-OTHER-WORLDS.png'

In [1091]:
added = []


In [1092]:
def generate_page_and_images(files_folder, write=False):
    images = find_all_images(files_folder)
    subfolder = files_folder.rsplit('/', 2)[-1]
    log.debug(f"subfolder {subfolder}")
    os.makedirs(outfolder + subfolder, exist_ok=1)

    div = ET.Element("div", attrib={"class":"rtecenter", "style":"display:flex;flex-wrap:wrap;align-items:center;justify-content: space-around"})
#     div = ET.Element("div", attrib={"class":"rtecenter", "style":"display:table;"})
    
    for image in images:
        file = pathlib.Path(image)
        
        myover = overrides[file.name] if file.name in overrides.keys() else  {}
        
        
            
            
        
        
        log.debug(f"image {image}")
        name = os.path.split(image)[-1]
        filename =sanitize_filepath (name).replace(" ", "_")
        filename_thumb = filename[:-4] + ".thumb.jpg"

        outfile = os.path.join(outfolder, subfolder, filename)
        outfile_thumb = os.path.join(outfolder, subfolder, filename_thumb)
        log.debug(outfile)
        resize_image(image, outfile, np.array((2048, 2048)), write=write)
        resize_image(image, outfile_thumb, np.array((2048, 2048))/4, write=write, toformat="jpg")
        
        
        
        url = os.path.join(dataurl, subfolder, filename)
        url = url[:-3] + "png" if "href" not in myover else myover["href"]
        url_thumb = os.path.join(dataurl, subfolder, filename_thumb)
        local_url_thumb = os.path.join(outfolder, subfolder, filename_thumb)
        url_thumb = url_thumb[:-3] + "jpg"
        local_url_thumb = local_url_thumb[:-3] + "jpg"
        log.debug(url)
        log.debug(url_thumb)
        
        figure = ET.Element("figure", attrib={"style": "display: table-cell;flex: 1 0 auto;"})
        caption = ET.Element("figcaption", attrib={"style": "display: table-caption;caption-side: bottom; width:auto;"})
        
        got = init_df.loc[init_df["fname"] == file.name, "caption"].values[0]
        if  isinstance(got, str):
            mycap = got
        else:
            mycap = None
        
        if mycap is not None:
            caption.text = mycap

        link = ET.Element("a", attrib={"href": url, "style": "display: table-cell;flex: 1 0 auto;"})
       
        img = ET.Element("img", attrib={"alt":f"Broken link: {name}", "src": url_thumb, "style": "border-radius:2%; align:center;  object-fit: contain; max-height:250px; width:auto; "})
         
#         link.append(figure)
        link.append(img)
        
        figure.append(link)
#         figure.append(img)
        if mycap is not None:
            figure.append(caption)
        div.append(figure)
        def path_to_image_html(path):
            return '<img src="'+ path + '" width="60" >'
        
        added.append({"fullpath": file,"fname": file.name, "author": file.parent.name, "thumb_url": url_thumb})
        
    asstring = ET.tostring(div, encoding='unicode', method='xml')

#     ET.ElementTree(div).write(os.path.join(outfolder, f"{subfolder}.html"))
    
    return asstring
style="height:725px;max-width:500px;width: expression(this.width > 500 ? 500: true);"
attrib={"style": "display: inline-block;"}

In [1093]:
out  = {}

for name in ["bianca", "jacopo", "carlotta"]:
    folder = "/home/luca/Documenti/planmap/website_data/original/" + name
    log.info(f"working in {folder}")
    out[name] = generate_page_and_images(folder, write=False)
    
    

INFO:root:working in /home/luca/Documenti/planmap/website_data/original/bianca
INFO:root:working in /home/luca/Documenti/planmap/website_data/original/jacopo
INFO:root:working in /home/luca/Documenti/planmap/website_data/original/carlotta


In [1094]:
from jinja2 import Template

template = "./artists.tpl"
outfile = template[:-3] + "html"

ll = open(template, 'r').read()

t = Template(ll)
rendered = t.render(out)


with open(outfile, 'w') as f:
    f.write(rendered)

In [1095]:
df = pandas.DataFrame(added)
# df.loc[:, "image"] = ""
# df.loc[:, "caption"] = ""
# df.loc[:, "alt_href"] = ""


In [1096]:
init_df.set_index("fname", inplace=True, drop=True)
df.set_index("fname", inplace=True, drop=True)

init_df.update(df, overwrite=False)
df = init_df

In [1097]:
# init_df.update?

In [1098]:
df["thumb_url"] = df["thumb_url"].apply(lambda x: f'=image("{x}")')

In [1099]:
df

,fullpath,author,thumb_url,caption,alt_href
fname,,,,,
0 -PLaNCK-PLANMAP_GEOMAPPING-OTHER-WORLDS.png,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,https://www.planck-magazine.it/le-uscite/?tito...
Presentazione MARIE_inglese.png,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN
Presentazione MAX_inglese.png,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN
page_1.jpg,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN
page_2.jpg,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN
page_3.jpg,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN
page_4.png,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN
page_5.png,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN
page_6.png,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN


In [1100]:
# # Create a Pandas Excel writer using XlsxWriter as the engine.
# writer = pandas.ExcelWriter(excel_file, engine='xlsxwriter')

# # Convert the dataframe to an XlsxWriter Excel object.
# df.to_excel(writer, sheet_name='Sheet1')

# # Get the xlsxwriter workbook and worksheet objects.
# workbook  = writer.book
# worksheet = writer.sheets['Sheet1']

# colid = df.columns.values.tolist().index("image") 
# col = chr(colid + 66)
# tmpdf = df.reset_index()
# for im in added:
#     fname = im["fname"]
#     local = im["thumb_local"]
#     row_id = tmpdf[tmpdf["fname"] == fname].index.values[0] + 2

#     # Insert an image.
#     excel_pos = f"{col}{row_id}"
# #     print(excel_pos)
#     s = 0.1
#     worksheet.insert_image(excel_pos, local, options={"x_scale":s, "y_scale": s} )

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()

In [1101]:
# options={"x_scale":s, "y_scale": s}
# options.get("x_scale")

In [1102]:
df

,fullpath,author,thumb_url,caption,alt_href
fname,,,,,
0 -PLaNCK-PLANMAP_GEOMAPPING-OTHER-WORLDS.png,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,https://www.planck-magazine.it/le-uscite/?tito...
Presentazione MARIE_inglese.png,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN
Presentazione MAX_inglese.png,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN
page_1.jpg,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN
page_2.jpg,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN
page_3.jpg,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN
page_4.png,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN
page_5.png,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN
page_6.png,/home/luca/Documenti/planmap/website_data/orig...,bianca,"=image(""0"")",NaN,NaN


In [1103]:
df.to_excel(excel_file)

In [1104]:
# t.render(out)